In [ ]:
%%capture
!pip install langchain==0.1.4 openai==1.10.0 langchain-openai datasets

In [ ]:
import os
import getpass

os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter Your OpenAI API Key:")

# FewShotPromptTemplate


🎯 **FewShotPromptTemplate Overview**: Tailor the AI's approach with examples for better task performance.

📚 **Use Cases**:

- **Context-Heavy Tasks**: Like giving a brief for a specific project.

- **Style/Tone Setting**: Directing the AI's "voice" with examples.

> 📝 **FewShotPromptTemplate**: Prime the model with I/O samples to steer its approach.

### Using FewShotPromptTemplate:

1. **Create Examples**: Make a list of example dicts with "input" and "output".

2. **Formatter Setup**: Design how these examples should be presented.

3. **Template Assembly**: Combine examples with the formatter in the template.

### Key Features:

- 🧩 **Tailored Examples**: Show the AI the specific input/output you expect.

- 🌐 **Context Inclusion**: Give examples that add relevant background.

- ✍️ **Styled Prompts**: Format your main prompt to echo the examples' style.

This template primes the model more effectively, ensuring a nuanced response tailored to the task's specifics.

In [7]:
from langchain.prompts import FewShotPromptTemplate

# Examples that demonstrate the desired input/output format
examples = [
    {
        "question": "Who rocked the fashion scene better, Cleopatra or Queen Elizabeth I?",
        "answer": (
            "Is a fashion runway showdown needed here: Totally!\n"
            "Follow up: Describe Cleopatra's iconic style.\n"
            "Intermediate answer: Cleopatra was known for her kohl-lined eyes and intricate jewelry.\n"
            "Follow up: And Queen Elizabeth I's fashion statement?\n"
            "Intermediate answer: Elizabeth I was all about the ruffs, pearls, and powdered wigs.\n"
            "Final Verdict: Cleopatra, with her exotic allure, narrowly takes the crown!"
        )
    },
    {
        "question": "When did the creator of Myspace join the internet party?",
        "answer": (
            "Ready for a digital time travel? Let's dive!\n"
            "Follow up: Who started Myspace?\n"
            "Intermediate answer: Myspace was co-founded by Chris DeWolfe and Tom Anderson.\n"
            "Follow up: When did they launch their masterpiece?\n"
            "Intermediate answer: Myspace was introduced to the world in 2003.\n"
            "Final Verdict: 2003 - When Myspace became the internet's coolest hangout."
        )
    },
    {
        "question": "Who was the great-grandfather of Winston Churchill?",
        "answer": (
            "Going down the ancestry rabbit hole, are we?\n"
            "Follow up: Who was Winston's father?\n"
            "Intermediate answer: Winston Churchill's father was Lord Randolph Churchill.\n"
            "Follow up: And who was Lord Randolph's grandpa?\n"
            "Intermediate answer: That would be George Spencer-Churchill, the 5th Duke of Marlborough.\n"
            "Final Verdict: George Spencer-Churchill is the answer, no doubt about it!"
        )
    },
    {
        "question": "Do the creators of 'Mario' and 'Sonic' hail from the same land?",
        "answer": (
            "Ready for a gaming face-off?\n"
            "Follow up: Who's the genius behind 'Mario'?\n"
            "Intermediate Answer: 'Mario' was crafted by Shigeru Miyamoto of Nintendo.\n"
            "Follow up: And from where does Miyamoto hail?\n"
            "Intermediate Answer: Japan, the land of sushi and sumo!\n"
            "Follow up: And the mastermind behind 'Sonic'?\n"
            "Intermediate Answer: 'Sonic' was created by Yuji Naka for Sega.\n"
            "Follow up: And Yuji Naka's home country?\n"
            "Intermediate Answer: Also Japan, where gaming legends are born!\n"
            "Final Verdict: Undoubtedly, both legends emerged from Japan."
        )
    }
]

In [8]:
from langchain.prompts.prompt import PromptTemplate

example_prompt = PromptTemplate.from_template(template="Question: {question}\n{answer}")

In [9]:
# Feed examples and formatter to FewShotPromptTemplate

prompt = FewShotPromptTemplate(
    examples=examples,
    example_prompt=example_prompt,
    suffix="Question: {input}",
    input_variables=["input"]
)

print(prompt.format(input="Who had better taste Steve Jobs or Bill Gates?"))

Question: Who rocked the fashion scene better, Cleopatra or Queen Elizabeth I?
Is a fashion runway showdown needed here: Totally!
Follow up: Describe Cleopatra's iconic style.
Intermediate answer: Cleopatra was known for her kohl-lined eyes and intricate jewelry.
Follow up: And Queen Elizabeth I's fashion statement?
Intermediate answer: Elizabeth I was all about the ruffs, pearls, and powdered wigs.
Final Verdict: Cleopatra, with her exotic allure, narrowly takes the crown!

Question: When did the creator of Myspace join the internet party?
Ready for a digital time travel? Let's dive!
Follow up: Who started Myspace?
Intermediate answer: Myspace was co-founded by Chris DeWolfe and Tom Anderson.
Follow up: When did they launch their masterpiece?
Intermediate answer: Myspace was introduced to the world in 2003.
Final Verdict: 2003 - When Myspace became the internet's coolest hangout.

Question: Who was the great-grandfather of Winston Churchill?
Going down the ancestry rabbit hole, are we

In [10]:
# Pass this to an llm
from langchain_openai import ChatOpenAI

from langchain_core.output_parsers import StrOutputParser

llm = ChatOpenAI(model="gpt-3.5-turbo-1106", temperature=0.1)

chain = prompt | llm | StrOutputParser()

for chunk in chain.stream({"input":"Who had better fashion taste Steve Jobs or Bill Gates?"}):
  print(chunk, end="", flush=True)

Is a tech mogul fashion face-off in order? Absolutely!
Follow up: Describe Steve Jobs' iconic style.
Intermediate answer: Steve Jobs was known for his signature black turtleneck, jeans, and New Balance sneakers.
Follow up: And Bill Gates' fashion statement?
Intermediate answer: Bill Gates is known for his casual and often nerdy style, often seen in khakis and a sweater.
Final Verdict: Steve Jobs, with his minimalist and iconic style, takes the fashion crown!

# Let's engineer this prompt

🔧 **Prompt Engineering with LLMs**: As AI grows smarter, we’re feeding it a wider variety of examples.

📊 **Challenge**: More classes mean more examples to handpick, which can be tough.

🎲 **Current Solution**: We often rely on randomly picking from a set of examples.

Let’s engineer a prompt that handles class diversity smartly and scales with ease.

---

We'll use this [dataset](https://huggingface.co/datasets/Bhuvaneshwari/intent_classification)

In [ ]:
%%capture
from datasets import load_dataset
dataset = load_dataset("Bhuvaneshwari/intent_classification", split='train')

In [11]:
intent_dataset = dataset.to_pandas()
intent_dataset.head()

,text,intent
0,listen to westbam alumb allergic on google music,PlayMusic
1,add step to me to the 50 clásicos playlist,AddToPlaylist
2,i give this current textbook a rating value of...,RateBook
3,play the song little robin redbreast,PlayMusic
4,please add iris dement to my playlist this is ...,AddToPlaylist


In [12]:
import random
from typing import List, Dict, Union
import pandas as pd

def get_prefix(data: Union[List[Dict[str, str]], 'pd.DataFrame'], prefix: str = "") -> str:
    """
    Generate a prefix string for the prompt to guide the language model in intent selection.

    The function takes a dataset (which can be a list of dictionaries or a pandas DataFrame)
    with 'text' and 'intent' columns, and an optional prefix string. It returns a formatted string
    that lists all unique intents from the dataset, prefixed with the provided prefix.

    Parameters:
    - data: List or a pandas DataFrame containing text and its associated intent.
    - prefix: Optional string to be prepended to the result.

    Returns:
    - str: A formatted string listing all unique intents from the dataset.
    """
    # Extract unique intents from the dataset
    if isinstance(data, list):
        unique_intents = set(item['intent'] for item in data)
    else:
        unique_intents = data.intent.unique()

    # Format the prefix, list of intents, and other parts of the string
    prompt = f"{prefix}\n" if prefix else ""
    prompt += f"Choose from the following intents: {', '.join(unique_intents)}"

    return prompt

In [13]:
print(get_prefix(intent_dataset, "Each input is associated with a user intent."))

Each input is associated with a user intent.
Choose from the following intents: PlayMusic, AddToPlaylist, RateBook, Greetings, SearchScreeningEvent, BookRestaurant, GetWeather, Book Meeting, SearchCreativeWork, Cancellation, Affirmation, excitment


In [14]:
def get_examples(data: Union[List[Dict[str, str]], 'pd.DataFrame']) -> List[Dict[str, str]]:
    """
    Extract one random example from each unique intent present in the dataset.

    The function processes a dataset (which can be a list of dictionaries or a pandas DataFrame)
    with 'text' and 'intent' columns. It returns a list containing one random example for each
    unique intent in the dataset.

    Parameters:
    - data: Dataset containing text and its associated intent.

    Returns:
    - List[Dict[str, str]]: A list of dictionaries, each containing a 'text' and 'intent' pair.
    """

    # Set a seed for reproducibility
    random.seed(42)

    # Group the data by 'intent', then take a random sample from each group
    samples = data.groupby('intent').apply(lambda group: group.sample(n=1)).reset_index(drop=True)

    # Convert the sampled data to a list of dictionaries
    examples = samples.to_dict(orient='records')

    return examples

# Using the function to get examples from the dataset
examples_dataset = get_examples(intent_dataset)
examples_dataset

[{'text': 'add this roy orbison song onto women of comedy',
  'intent': 'AddToPlaylist'},
 {'text': 'Of Course!', 'intent': 'Affirmation'},
 {'text': 'arrange a meeting ', 'intent': 'Book Meeting'},
 {'text': 'i want to book a delicatessen serving testaroli in somalia for 7/25/2027',
  'intent': 'BookRestaurant'},
 {'text': 'cancel this', 'intent': 'Cancellation'},
 {'text': 'what s the weather forecast close by sligo',
  'intent': 'GetWeather'},
 {'text': 'Good Evening!', 'intent': 'Greetings'},
 {'text': 'play a symphonic rock on pandora', 'intent': 'PlayMusic'},
 {'text': 'rate the current saga one stars', 'intent': 'RateBook'},
 {'text': 'can i hear the song visual audio sensory theater',
  'intent': 'SearchCreativeWork'},
 {'text': 'where is walt: the man behind the myth playing',
  'intent': 'SearchScreeningEvent'},
 {'text': 'Fabulous', 'intent': 'excitment'}]

In [15]:
from typing import List, Dict, Union

def construct_prompt(data: Union[List[Dict[str, str]], 'pd.DataFrame'],
               text: str,
               examples: List[Dict[str, str]],
               prefix: str = "") -> str:
    """
    Construct a formatted k-shot prompt using:
    - A prefix.
    - Specified examples (one from each intent).
    - Target text for classification.

    The function takes a dataset, a target text, examples, and an optional prefix. It then uses these
    to craft a structured k-shot prompt using the PromptTemplate and FewShotPromptTemplate classes.

    Parameters:
    - data: Dataset containing text and its associated intent.
    - text: Target text to be classified.
    - examples: List of dictionaries, each containing a 'text' and 'intent' pair.
    - prefix: Optional string to be prepended to the prompt.

    Returns:
    - str: Formatted k-shot prompt.
    """

    # Define the prompt template with specified format
    prompt_template = PromptTemplate(
        input_variables=["text", "intent"],
        template="Text: {text}\nLabel: {intent}",
    )

    # Construct the few-shot prompt using the provided examples, prefix, and other configurations
    prompt = FewShotPromptTemplate(
        example_prompt = prompt_template,
        examples = examples,
        prefix = get_prefix(data, prefix),
        suffix = "Text: {text}\nLabel:",
        input_variables = ['text'],
    )

    # Return the formatted prompt
    return prompt.format(text=text).strip()

# Example usage

prompt = construct_prompt(intent_dataset,
                 "Save Free Smoke by AP Dhillon to my songs",
                 examples_dataset,
                 "Each input belongs is associated with a user intent")

print(prompt)

Each input belongs is associated with a user intent
Choose from the following intents: PlayMusic, AddToPlaylist, RateBook, Greetings, SearchScreeningEvent, BookRestaurant, GetWeather, Book Meeting, SearchCreativeWork, Cancellation, Affirmation, excitment

Text: add this roy orbison song onto women of comedy
Label: AddToPlaylist

Text: Of Course!
Label: Affirmation

Text: arrange a meeting 
Label: Book Meeting

Text: i want to book a delicatessen serving testaroli in somalia for 7/25/2027
Label: BookRestaurant

Text: cancel this
Label: Cancellation

Text: what s the weather forecast close by sligo
Label: GetWeather

Text: Good Evening!
Label: Greetings

Text: play a symphonic rock on pandora
Label: PlayMusic

Text: rate the current saga one stars
Label: RateBook

Text: can i hear the song visual audio sensory theater
Label: SearchCreativeWork

Text: where is walt: the man behind the myth playing
Label: SearchScreeningEvent

Text: Fabulous
Label: excitment

Text: Save Free Smoke by AP Dh

In [16]:
# Pass this to an llm
from langchain_openai import ChatOpenAI

from langchain_core.output_parsers import StrOutputParser

llm = ChatOpenAI(model="gpt-3.5-turbo-1106", temperature=0.75)

response = llm.invoke(prompt)

print(response.content)

AddToPlaylist
